In [47]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

Inspect the indeed dataset. 

In [6]:
indeed_data = pd.read_csv("data_sub.csv").drop(["Unnamed: 0"], axis = 1)

In [7]:
indeed_data.head()

,date,companyId,jobId,country,stateProvince,city,avgOverallRating,numReviews,industry,normTitle,...,experienceRequired,estimatedSalary,salaryCurrency,jobLanguage,supervisingJob,licenseRequiredJob,educationRequirements,jobAgeDays,clicks,localClicks
0,2016-11-01,company00194,job0000208,US,TX,Odessa,0.0,NaN,NaN,security officer,...,NaN,18700,NaN,EN,0.0,1.0,High School,98,19,0
1,2016-11-01,company00262,job0000278,DE,SN,Crimmitschau,0.0,NaN,NaN,einrichter,...,NaN,26700,NaN,DE,NaN,NaN,NaN,97,20,0
2,2016-11-02,company00262,job0000278,DE,SN,Crimmitschau,0.0,NaN,NaN,einrichter,...,NaN,26700,NaN,DE,NaN,NaN,NaN,98,23,2
3,2016-11-03,company00262,job0000278,DE,SN,Crimmitschau,0.0,NaN,NaN,einrichter,...,NaN,26700,NaN,DE,NaN,NaN,NaN,99,25,1
4,2016-11-01,company00326,job0000344,US,VA,Vienna,0.0,NaN,NaN,deputy manager,...,7.0,73500,NaN,EN,1.0,0.0,Higher Education,97,27,2


In [8]:
len(indeed_data)

138297

In [9]:
indeed_data.describe()

,avgOverallRating,numReviews,descriptionCharacterLength,descriptionWordCount,experienceRequired,estimatedSalary,supervisingJob,licenseRequiredJob,jobAgeDays,clicks,localClicks
count,138297.000000,60406.000000,138297.000000,138297.000000,50437.000000,138297.000000,116845.000000,116845.000000,138297.000000,138297.000000,138297.000000
mean,1.717282,272.098186,2396.075808,388.495275,3.368010,43470.596615,0.159194,0.358432,30.348026,22.606224,3.646211
std,1.888626,1155.468292,1741.108796,251.228217,3.124506,28499.397923,0.365858,0.479542,25.507355,44.276172,9.664703
min,0.000000,2.000000,27.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,39.000000,1176.000000,210.000000,1.700000,23600.000000,0.000000,0.000000,9.000000,10.000000,0.000000
50%,0.000000,70.000000,1971.000000,330.000000,2.000000,33500.000000,0.000000,0.000000,23.000000,18.000000,1.000000
75%,3.700000,103.000000,3113.000000,501.000000,5.000000,55500.000000,0.000000,1.000000,47.000000,25.000000,3.000000
max,5.000000,10565.000000,14976.000000,2228.000000,20.000000,235500.000000,1.000000,1.000000,100.000000,6174.000000,708.000000


In [10]:
indeed_data.columns

Index(['date', 'companyId', 'jobId', 'country', 'stateProvince', 'city',
       'avgOverallRating', 'numReviews', 'industry', 'normTitle',
       'normTitleCategory', 'descriptionCharacterLength',
       'descriptionWordCount', 'experienceRequired', 'estimatedSalary',
       'salaryCurrency', 'jobLanguage', 'supervisingJob', 'licenseRequiredJob',
       'educationRequirements', 'jobAgeDays', 'clicks', 'localClicks'],
      dtype='object')

In [12]:
len(indeed_data.city.unique())

2050

In [13]:
len(indeed_data.stateProvince.unique())

78

We chose to scrape data on the cost of living in various areas of the US.

In [24]:
# Get the request and webscrape the table
req = requests.get("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2017&region=019")
soup = BeautifulSoup(req.text, "html.parser")

In [43]:
col_columns = []

col_table = soup.find_all("table", {"id":"t2"})[0]
col_header = col_table.find("thead")
col_body = col_table.find("tbody")

In [68]:
for th in col_header.find_all("th"):
    col_columns.append(th.text)

In [53]:
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

In [64]:
first = True
rows = []
rank = 1
for tr in col_body.find_all("tr"):
    rows.append([rank] + tr.text.split("\n")[2:-1])
    rank += 1


In [71]:
col_city = pd.DataFrame(data = rows, columns=col_columns)

After scraping data on a couple of areas, we instead chose to scrape data on an index of the cost of living in each state of the US based on 2018 data. 

In [72]:
# Get the request and webscrape the table
req = requests.get("https://www.missourieconomy.org/indicators/cost_of_living/")
soup = BeautifulSoup(req.text, "html.parser")

In [100]:
col_table = soup.find("table", {"class": "excel28"})

In [109]:
columns = []
rows = []
count = 0

for tr in col_table.find_all("tr"):
    if count >= 2:
        rows.append(tr.text.split("\n")[1:-1])
    elif count == 1:
        columns.append(tr.text.split("\n"))
    count += 1

In [110]:
columns = columns[0][1:-1]

In [111]:
columns = [val.strip("\xa0") for val in columns]

In [114]:
col_by_state = pd.DataFrame(data = rows, columns = columns)

In [118]:
col_by_state[col_by_state.columns[1:]] = col_by_state[col_by_state.columns[1:]]\
.apply(pd.to_numeric, errors = 'coerce')